In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import caffe
from pylab import *
plt.rcParams['figure.figsize']=(10,10)
plt.rcParams['image.interpolation']='nearest'
plt.rcParams['image.cmap']='gray'

In [2]:
caffe.set_mode_cpu()
net=caffe.Net('D:\CaffeNet\caffe\deploy.prototxt','D:\CaffeNet\caffe\cifar_48.caffemodel',caffe.TEST)

In [3]:
transformer=caffe.io.Transformer({'data':net.blobs['data'].data.shape})
transformer.set_transpose('data',(2,0,1))
transformer.set_mean('data',np.load('D:\CaffeNet\ilsvrc_2012_mean.npy').mean(1).mean(1))
transformer.set_raw_scale('data',255)
transformer.set_channel_swap('data',(2,1,0))

In [4]:
net.blobs['data'].data[...]=transformer.preprocess('data',caffe.io.load_image('D:\CaffeNet\images\cat.jpg'))
out=net.forward()
print("Predicted class is #{}.".format(out['prob'].argmax()))

KeyError: 'prob'

In [5]:
plt.imshow(transformer.deprocess('data',net.blobs['data'].data[0]))
plt.show()

In [21]:
imagenet_labels_filename='D:\CaffeNet\synset_words.txt'
try:
    labels=np.loadtxt(imagenet_labels_filename,str,delimiter='\t')
except:
    labels=np.loadtxt(imagenet_labels_filename,str,delimiter='\t')
top_k=net.blobs['prob'].data[0].flatten().argsort()[-1:-6:-1]
print labels[top_k]

['n04461696 tow truck, tow car, wrecker'
 'n04467665 trailer truck, tractor trailer, trucking rig, rig, articulated lorry, semi'
 'n04065272 recreational vehicle, RV, R.V.'
 'n04252225 snowplow, snowplough' 'n04252077 snowmobile']


In [9]:
net.forward()

{'fc8_kevin_encode': array([[  9.99952376e-01,   3.15452665e-01,   2.58057564e-01,
           8.85128260e-01,   2.67958850e-01,   9.98764932e-01,
           5.89062631e-01,   2.76432991e-01,   3.22964421e-04,
           9.99344468e-01,   2.70132899e-01,   2.21212864e-01,
           9.93147910e-01,   2.32841820e-01,   2.01567069e-01,
           6.67330444e-01,   7.56207630e-02,   9.38837469e-01,
           1.32468967e-02,   9.25138831e-01,   1.08206943e-01,
           3.98103222e-02,   6.32906109e-02,   3.75769855e-06,
           3.53849168e-06,   6.29283488e-03,   9.76623237e-01,
           7.67149329e-02,   9.97754335e-01,   9.83976424e-01,
           9.99860585e-01,   1.00216847e-02,   9.06890929e-01,
           1.70457326e-02,   8.68340861e-03,   4.82886210e-02,
           4.59896633e-03,   9.93868291e-01,   9.99455333e-01,
           9.95941460e-01,   9.99359667e-01,   9.59710896e-01,
           2.93468740e-02,   7.43153170e-02,   9.99132216e-01,
           7.29824305e-02,   9.7234

In [10]:
[(k, v.data.shape) for k, v in net.blobs.items()]

[('data', (10L, 3L, 227L, 227L)),
 ('conv1', (10L, 96L, 55L, 55L)),
 ('pool1', (10L, 96L, 27L, 27L)),
 ('norm1', (10L, 96L, 27L, 27L)),
 ('conv2', (10L, 256L, 27L, 27L)),
 ('pool2', (10L, 256L, 13L, 13L)),
 ('norm2', (10L, 256L, 13L, 13L)),
 ('conv3', (10L, 384L, 13L, 13L)),
 ('conv4', (10L, 384L, 13L, 13L)),
 ('conv5', (10L, 256L, 13L, 13L)),
 ('pool5', (10L, 256L, 6L, 6L)),
 ('fc6', (10L, 4096L)),
 ('fc7', (10L, 4096L)),
 ('fc8_kevin', (10L, 48L)),
 ('fc8_kevin_encode', (10L, 48L))]

In [11]:
[(k, v[0].data.shape) for k, v in net.params.items()]

[('conv1', (96L, 3L, 11L, 11L)),
 ('conv2', (256L, 48L, 5L, 5L)),
 ('conv3', (384L, 256L, 3L, 3L)),
 ('conv4', (384L, 192L, 3L, 3L)),
 ('conv5', (256L, 192L, 3L, 3L)),
 ('fc6', (4096L, 9216L)),
 ('fc7', (4096L, 4096L)),
 ('fc8_kevin', (48L, 4096L))]

In [6]:
# take an array of shape (n, height, width) or (n, height, width, channels)
# and visualize each (height, width) thing in a grid of size approx. sqrt(n) by sqrt(n)
def vis_square(data, padsize=1, padval=0):
    data -= data.min()
    data /= data.max()
    
    # force the number of filters to be square
    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, padsize), (0, padsize)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant', constant_values=(padval, padval))
    
    # tile the filters into an image
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    
    plt.imshow(data)

In [7]:
# the parameters are a list of [weights, biases]
filters = net.params['conv1'][0].data
vis_square(filters.transpose(0, 2, 3, 1))

In [21]:
plt.show()

In [16]:
feat = net.blobs['conv1'].data[0, :36]
vis_square(feat, padval=1)

In [11]:
filters = net.params['conv2'][0].data
vis_square(filters[:48].reshape(48**2, 5, 5))

In [13]:
feat = net.blobs['conv2'].data[0, :36]
vis_square(feat, padval=1)

In [34]:
feat = net.blobs['conv3'].data[0]
vis_square(feat, padval=0.5)

In [36]:
feat = net.blobs['conv4'].data[0]
vis_square(feat, padval=0.5)

In [37]:
feat = net.blobs['conv5'].data[0]
vis_square(feat, padval=0.5)

In [15]:
feat = net.blobs['pool5'].data[0]
vis_square(feat, padval=1)

In [17]:
feat = net.blobs['fc6'].data[0]
plt.subplot(2, 1, 1)
plt.plot(feat.flat)
plt.subplot(2, 1, 2)
_ = plt.hist(feat.flat[feat.flat > 0], bins=100)

In [20]:
feat = net.blobs['fc7'].data[0]
plt.subplot(2, 1, 1)
plt.plot(feat.flat)
plt.subplot(2, 1, 2)
_ = plt.hist(feat.flat[feat.flat > 0], bins=100)

In [22]:
feat = net.blobs['prob'].data[0]
plt.plot(feat.flat)

KeyError: 'prob'